# ตัวอย่างปัญหากำหนดการเชิงเส้น (Linear programming problem)

<img src="Image/LP.png">

In [47]:
import json

with open("Problem/LPProblem.txt") as f:
    # Read data
    data = f.read()
    # JSON format
    LPProblem = json.loads(data)
    
    # Set of variable names
    S_VarsName = LPProblem['S_VarsName']
    print('S_VarsName',S_VarsName)
    
    # Objective function parameter
    P_Cost = LPProblem['P_Cost']
    print('P_Cost',P_Cost)
    
    # Objective function parameter
    P_Constraint = LPProblem['P_Constraint']
    print('P_Constraint',P_Constraint)
    
    # Objective function parameter
    P_RHS = LPProblem['P_RHS']
    print('P_RHS',P_RHS)

S_VarsName ['x0', 'x1']
P_Cost {'c0': 40, 'c1': 50}
P_Constraint {'a00': 1, 'a01': 2, 'a10': 4, 'a11': 3}
P_RHS {'b0': 40, 'b1': 120}


# การสร้าง Optimization Model ด้วย Pyomo 

In [49]:
import pyomo.environ as pyo

model = pyo.ConcreteModel()

# Model variables
model.var = pyo.Var([varName for varName in S_VarsName],domain=pyo.NonNegativeReals)

# Objective function
model.OBJ = pyo.Objective(expr = P_Cost['c0']*model.var['x0'] + P_Cost['c1']*model.var['x1'], sense=pyo.maximize)

# Constraint function
model.Constraint1 = pyo.Constraint(expr = P_Constraint['a00']*model.var['x0'] + P_Constraint['a01']*model.var['x1'] <= P_RHS['b0'])
model.Constraint2 = pyo.Constraint(expr = P_Constraint['a10']*model.var['x0'] + P_Constraint['a11']*model.var['x1'] <= P_RHS['b1'])

model.pprint()

1 Set Declarations
    var_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'x0', 'x1'}

1 Var Declarations
    var : Size=2, Index=var_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
         x0 :     0 :  None :  None : False :  True : NonNegativeReals
         x1 :     0 :  None :  None : False :  True : NonNegativeReals

1 Objective Declarations
    OBJ : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 40*var[x0] + 50*var[x1]

2 Constraint Declarations
    Constraint1 : Size=1, Index=None, Active=True
        Key  : Lower : Body                : Upper : Active
        None :  -Inf : var[x0] + 2*var[x1] :  40.0 :   True
    Constraint2 : Size=1, Index=None, Active=True
        Key  : Lower : Body                  : Upper : Active
        None :  -Inf : 4*var[x0] + 3*var[x1] : 120.0 :   True

5 Declarations: var_index 

# Solve the model

In [51]:
pyo.SolverFactory("cbc").solve(model, timelimit=None).write() # timelimit (secs)

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: -1360.0
  Upper bound: -1360.0
  Number of objectives: 1
  Number of constraints: 3
  Number of variables: 3
  Number of nonzeros: 2
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number of created

# ผลลัพธ์ของ Solver (Optimal solution)

In [52]:
optimalValue = model.OBJ()
optimalSolution = {varName:model.var[varName].value for varName in S_VarsName}
print(f"Optimal value : {optimalValue}\n"
      f"Optimal solution : {optimalSolution}")

Optimal value : 1360.0
Optimal solution : {'x0': 24.0, 'x1': 8.0}
